In [1]:
DAY = "24"

In [2]:
import os
with open(f'session.secret','r') as f:
    SESSION = f"session={f.read()}"

INPUT_FILE = f"{DAY}.txt"

In [3]:
# Manual INPUT
INPUT ="""x00: 1
x01: 1
x02: 1
y00: 0
y01: 1
y02: 0

x00 AND y00 -> z00
x01 XOR y01 -> z01
x02 OR y02 -> z02
"""

In [4]:
# Automatic INPUT
import os.path
if not os.path.isfile(INPUT_FILE):
    !curl https://adventofcode.com/2024/day/{DAY}/input --cookie {SESSION} > {INPUT_FILE}
with open(INPUT_FILE,'r') as f:
    INPUT = f.read().strip()

## Usefull import

In [5]:
import numpy as np
import pandas as pd
import networkx as nx
import copy as cp
import re

In [6]:
DIR = {(0,1):'>',(1,0):'v',(0,-1):'<',(-1,0):'^'}
DIRd = {v:k for k,v in DIR.items()}

## Solution

In [7]:
OP ={
    'OR':(lambda x1,x2: x1 or x2),
    'XOR':(lambda x1,x2: x1 != x2),
    'AND':(lambda x1,x2: x1 and x2)
}

In [8]:
def sort_circuit(circuit,init_vals):
    sorted_circuit = []
    vals= set(init_vals)
    while len(sorted_circuit)<len(circuit):
        for c in circuit:
            if c in sorted_circuit:
                continue
            if c[0] in vals and c[2] in vals:
                sorted_circuit.append(c)
                vals.add(c[3])
    return sorted_circuit

In [9]:
val = {row.split(':')[0].strip() : bool(int(row.split(':')[1].strip())) for row in INPUT.split('\n\n')[0].strip().split('\n')}
circuit = re.findall(r"([0-9a-z]{3}) (AND|XOR|OR) ([0-9a-z]{3}) -> ([0-9a-z]{3})",INPUT)
circuit = sort_circuit(circuit,val.keys())

In [10]:
for x1,oper,x2,res in circuit:
    val[res] = OP[oper](val[x1],val[x2])
z = 0
for var,value in val.items():
    if var[0]!='z':
        continue
    if value:
        digit = int(var[1:])
        z+= 1<<digit
z

58740594706150

In [11]:
val = {row.split(':')[0].strip() : bool(int(row.split(':')[1].strip())) for row in INPUT.split('\n\n')[0].strip().split('\n')}
circuit = re.findall(r"([0-9a-z]{3}) (AND|XOR|OR) ([0-9a-z]{3}) -> ([0-9a-z]{3})",INPUT)
circuit_dep = {c[3]:c[:3] for c in circuit}
def all_input_dependencies(output):
    if output not in circuit_dep:
        return {output}
    return all_input_dependencies(circuit_dep[output][0]) | all_input_dependencies(circuit_dep[output][2])


In [13]:
def tree(output,depth=-1):
    if output not in circuit_dep or depth==0:
        return output
    return f"{output}=({tree(circuit_dep[output][0],depth-1)} {circuit_dep[output][1]} {tree(circuit_dep[output][2],depth-1)})"

In [30]:
def new_part(p1,op,p2):
    """ Determine what part of the addition is the result of the operation.
        For example:
            (r,1) means the operation computes the reminder for the first digit
            (z,n) is the n-th digit of the addition result
            ...
    """
    x,dx = p1 if p1[0]<p2[0] else p2
    y,dy = p2 if p1[0]<p2[0] else p1
    if dx==0 and dy==0:
        if (x,y,op) == ('x','y','AND'):
            return ('r',1)
        if (x,y,op) == ('x','y','XOR'):
            return ('z',0)
    if (x,y,op)==('x','y','XOR') and dx==dy:
        return ('xoy',dx)
    if (x,y,op)==('x','y','AND') and dx==dy:
        return ('xay',dx)
    if (x,y,op)==('r','xoy','XOR') and dx==dy:
        return ('z',dx)
    if (x,y,op)==('r','xoy','AND') and dx==dy:
        return ('rxy',dx)
    if (x,y,op)==('rxy','xay','OR') and dx==dy:
        return ('r',dx+1) if dx+1<45 else ('z',dx+1)
    return None

In [33]:
def swapped_circuit(swap):
    circuit = {c[3]:list(c[:3]) for c in re.findall(r"([0-9a-z]{3}) (AND|XOR|OR) ([0-9a-z]{3}) -> ([0-9a-z]{3})",INPUT)}
    for o1,o2 in swap.items():
        circuit[o1],circuit[o2] = circuit[o2] ,circuit[o1]
    return circuit

def determine_all_parts(circuit):
    val = {row.split(':')[0].strip() : bool(int(row.split(':')[1].strip())) for row in INPUT.split('\n\n')[0].strip().split('\n')}
    parts = {x:(x[0],int(x[1:])) for x in val.keys()}
    changed = True
    while changed:
        changed=False
        for res,(x,op,y) in circuit.items():
            if x in parts and y in parts and res not in parts:
                part = new_part(parts[x],op,parts[y])
                if part is not None:
                    parts[res]=part
                    changed=True
    return parts

In [42]:
## My solution involved 
##     - run the "determine_all_parts" without any swap
##     - manually find the last zXX that doesn't match a part ('z',xx) 
##     - Manually find the swap to make it work
##     - Add the swap and then redo the steps until everything works
swap= {'z14':'hbk'
       ,'kvn':'z18'
       ,'z23':'dbb'
       ,'cvh':'tfn'}

parts = determine_all_parts(swapped_circuit(swap))

In [43]:
a = list(swap.keys()) + list(swap.items())
a.sort()
','.join(a)

TypeError: '<' not supported between instances of 'tuple' and 'str'

In [39]:
## automatic solution
is_correct = False
swap = dict()
while True: 
    sw_circuit = swapped_circuit(swap)
    parts = determine_all_parts(sw_circuit)
    
    stop = True
    for digit in range(46):
        z = f'z{digit:02}'
        z_part = ('z',digit)
        if z not in parts or parts[z]!=('z',digit):
            stop = False
            break
    if stop:
        break
    
    # z need to be swapped
    swapped = False
    for out,part in parts.items():
        if part==z_part:
            swap[z]=out
            swapped=True
            break
    if swapped:
        continue
    
    # depth 1 TODO
    sw_circuit[z]
    
    
    break
    
print(swap) 
a = list(swap.keys())
a.sort()
','.join(a)

{'z14': 'hbk', 'z18': 'kvn', 'z23': 'dbb'}


'z14,z18,z23'

In [17]:
tree('z34',depth=3)

'z34=(mqf=(cht=(y33 AND x33) OR mkv=(gtm AND rmt)) XOR cvh=(y34 AND x34))'

'cvh,dbb,hbk,kvn,tfn,z14,z18,z23'

In [36]:
sw_circuit[z]

['mqf', 'XOR', 'cvh']

KeyError: 'chv'